In [ ]:
#SBA practice   

In [ ]:
%pip install seaborn
%pip install folium

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
%matplotlib inline

In [ ]:
from js import fetch
import io

URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Historical_Wildfires.csv"
resp = await fetch(URL)
text = io.BytesIO((await resp.arrayBuffer()).to_py())
df = pd.read_csv(text)
print('Data read into a pandas dataframe!')


In [ ]:
df.head()

#Column names
df.columns

#data type
df.dtypes

In [ ]:
import datetime as dt

df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month


In [ ]:
#verify the columns again

df.columns #year and month are now included

In [ ]:
#TASK 1.1: Let's try to understand the change in average estimated fire area over time

# TODO

plt.figure(figsize=(12,6))
df_new = df.groupby('Year')['Estimated_fire_area'].mean()
df_new.plot(x=df_new.index, y=df_new.values)
plt.xlabel('Year')
plt.ylabel('Average Estimated Fire Area (km2)')
plt.title('Estimated Fire Area over Time')
plt.show()

In [ ]:
#TASK 1.2: You can notice the peak in the plot between 2010 to 2013. Let's narrow down our finding, by plotting the estimated fire area for year grouped together with month.

# TODO
#plt.figure(figsize=(12,6))
df_new2 = df.groupby(['Year', 'Month'])['Estimated_fire_area'].mean()
df_new2.plot(x=df_new2.index, y=df_new.values)
plt.xlabel('Year, Month')
plt.ylabel('Fire Area')
plt.title('Estimated Fire Area over time')
plt.show()

In [ ]:
#TASK 1.3: Let's have an insight on the distribution of mean estimated fire brightness across the regions
#use the functionality of seaborn to develop a barplot

df['Region'].unique()

#TODO
plt.figure(figsize=(10,6))
sns.barplot(data=df, x='Region', y='Mean_estimated_fire_brightness')
plt.xlabel('Region')
plt.ylabel('Mean Estimated Fire Brightness (Kelvin)')
plt.title('Distribution of Mean Estimated Fire Brightness across Regions')
plt.show()

In [ ]:
#TASK 1.4: Let's find the portion of count of pixels for presumed vegetation fires vary across regions
#we will develop a pie chart for this

plt.figure(figsize=(10,6))
region_counts = df.groupby('Region')['Count'].sum()
plt.pie(region_counts, labels=region_counts.index, autopct='%1.1f%%')
plt.title('Percentage of Pixels for Presumed Vegetation Fires by Region')
plt.axis('equal')
plt.show()

In [ ]:
#TASK 1.5: See the percentage on the pie is not looking so good as it is overlaped for Region SA, TA, VI
#remove the autopct fromm pie function and pass the following to plt.legend() after plt.title()

plt.figure(figsize=(10, 6))
region_counts = df.groupby('Region')['Count'].sum()
plt.pie(region_counts, labels=region_counts.index)
plt.title('Percentage of Pixels for Presumed Vegetation Fires by Region')
plt.legend([(i,round(k/region_counts.sum()*100,2)) for i,k in zip(region_counts.index, region_counts)])
plt.axis('equal')
plt.show()

In [ ]:
#TASK 1.6: Let's try to develop a histogram of the mean estimated fire brightness
#Using Matplotlib to create the histogram

plt.figure(figsize=(10, 6))
plt.hist(x=df['Mean_estimated_fire_brightness'], bins=20)
plt.xlabel('Mean Estimated Fire Brightness (Kelvin)')
plt.ylabel('Count')
plt.title('Histogram of Mean Estimated Fire Brightness')
plt.show()

In [ ]:
#TASK 1.7: What if we need to understand the distribution of estimated fire brightness across regions? 
#Let's use the functionality of seaborn and pass region as hue

plt.figure(figsize=(10,6))
sns.histplot(data=df, x='Mean_estimated_fire_brightness', hue="Region")
plt.show()

In [ ]:
#looks better!, now include the parameter multiple='stack' in the histplot() and see the difference. Include labels and titles as well

sns.histplot(data=df, x='Mean_estimated_fire_brightness', hue='Region', multiple='stack')
plt.show()

In [ ]:
#TASK 1.8: Let's try to find if there is any correlation between mean estimated fire radiative power and mean confidence level?

plt.figure(figsize=(8, 6))
sns.scatterplot(data=df, x='Mean_confidence', y='Mean_estimated_fire_radiative_power')
plt.xlabel('Mean Estimated Fire Radiative Power (MW)')
plt.ylabel('Mean Confidence')
plt.title('Mean Estimated Fire Radiative Power vs. Mean Confidence')
plt.show()

In [ ]:
#TASK 1.9: Let's mark these seven regions on the Map of Australia using Folium

#we have created a dataframe for you containing the regions, their latitudes and longitudes.
#For australia use [-25, 135] as location to create the map
region_data = {'region':['NSW','QL','SA','TA','VI','WA','NT'], 'Lat':[-31.8759835,-22.1646782,-30.5343665,-42.035067,-36.5986096,-25.2303005,-19.491411], 
               'Lon':[147.2869493,144.5844903,135.6301212,146.6366887,144.6780052,121.0187246,132.550964]}
reg=pd.DataFrame(region_data)
reg

# instantiate a feature group 
aus_reg = folium.map.FeatureGroup()

# Create a Folium map centered on Australia
Aus_map = folium.Map(location=[-25, 135], zoom_start=4)

# loop through the region and add to feature group
for lat, lng, lab in zip(reg.Lat, reg.Lon, reg.region):
    aus_reg.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            popup=lab,
            radius=5, # define how big you want the circle markers to be
            color='red',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
Aus_map.add_child(aus_reg)